In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score

import lightgbm as lgb

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
#tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
test_size = 48
x_test = x[-test_size:]
x = x[:-48]

y_test = y[-test_size:]
y = y[:-48]

print(x.shape)
print(x_test.shape)
print(y.shape)
print(y_test.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(densenet_type='DenseNet-121', base_growth_rate = 32):
    model_input = Input(shape=(28,28,1))
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    
    return model

In [9]:
def CNN():
    inputs = Input(shape=(28,28,1))
    x = Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    
    return model

In [10]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48]}

DN_model = DenseNet('DenseNet-121', 32)
CNN_model = CNN()

In [13]:
def get_stacking_data(model, x_train, y_train, x_test, n_folds=5):
    kfold = KFold(n_splits = n_folds)
    
    # 최종 모델에서 사용할 데이터셋 셋팅(0 값으로)
    # 만약 shape가 (100, 10) 이었으면 폴드의 검증 과정에서 저장할 데이터는 (100, 1)
    # 모양을 갖게 한다.
    train_fold_predict = np.zeros((x_train.shape[0], 1))
    
    # test는 x_test 값을 이용해서 매 폴드마다 예측을 하기 때문에 (100, fold개수)
    # 만큼의 shape를 갖게 된다.
    # 그래서 해당 폴드마다 x_test의 예측 값을 해당 fold에 해당되는 열에 넣는다.
    test_predict = np.zeros((x_test.shape[0], n_folds))
    print("model : ", model.__class__.__name__)
    
    for cnt, (train_index, valid_index) in enumerate(kfold.split(x_train)):
        x_train_ = x_train[train_index]
        y_train_ = y_train[train_index]
        x_validation = x_train[valid_index]
        
        x_train_ = x_train_.astype(np.float32)
        y_train_ = y_train_.astype(np.float32)
        x_validation = x_validation.astype(np.float32)
        # 학습
        model.fit(x_train_, y_train_, batch_size=64, epochs=30)
        
        # 해당 폴드에서 학습된 모델에다가 검증 데이터(x_validation)로 예측 후 저장
        train_fold_predict[valid_index, :] = model.predict(x_validation).reshape(-1,1)
        
        # 해당 폴드에서 생성된 모델에게 원본 테스트 데이터(x_test)를 이용해서 예측을 수행하고 저장
        test_predict[:, cnt] = model.predict(x_test)
    
    # for문이 끝나면 test_pred는 평균을 내서 하나로 합친다.
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    
    return train_fold_predict, test_predict_mean

In [14]:
DN_train, DN_test = get_stacking_data(DN_model, x_total, y_total, x_test)
CNN_train, CNN_test = get_stacking_data(CNN_model, x_total, y_total, x_test)

model :  Model
Train on 4800 samples
Epoch 1/30
4800/4800 [==============================] - 570s 119ms/sample - loss: 2.2648 - accuracy: 0.2775
Epoch 2/30
 896/4800 [====>.........................] - ETA: 8:22 - loss: 1.4211 - accuracy: 0.5012

KeyboardInterrupt: 

In [ ]:
new_x_train = np.concatenate((DN_train, CNN_train), axis=1)
new_x_test = np.concatenate((DN_test, CNN_test), axis=1)

print("원본 : ", x_total.shape, x_test.shape)
print("새로운 : ", new_x_train.shape, new_x_test.shape)

In [ ]:
#train_lgb = lgb.Dataset(new_x_train, label=y_total)
#params = {'random_seed':seed, 'bagging_seed':seed,
#          'feature_fraction_seed':seed, 'data_random_seed':seed,
#          'drop_seed':seed,
#          'num_iterations':400,
#          'boosting_type':'gbdt', 'objective':'regression_l1',
#          'learning_rate':0.05, 'num_leaves':100, 'max_depth':-1,
#          'bagging_fraction':0.1, 'feature_fraction':0.8,
#          'lambda_l1':0.0, 'lambda_l2':15.0, 'max_bin':300}

#lgbm = lgb.train(params, train_lgb, num_boost_round=2000, early_stopping_rounds=100,
                 verbose_eval=200)

In [ ]:
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = lgb.fit(new_x_train, y_train, callbacks=[cp])
stack_pred = lgbm.predict(new_x_test)

print(stack_pred)

In [ ]:
#from tensorflow.keras.models import load_model
#best_model = load_model('models/DenseNet121_submit.h5')

In [ ]:
#x_test = test[:,1:]
#x_test = np.reshape(x_test, (-1, 28, 28, 1))
#print(x_test.shape)

In [ ]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = np.argmax(best_model.predict(x_test), axis=1)
#submission.to_csv('data/submission_densenet(0821).csv', index=False)